# 零售客群分析

# Part 1. 编码值替换

In [34]:
import pandas as pd

# 读取KYC标签数据及其对应字典值表
customer_tags = pd.read_excel('/Users/suding/Desktop/Retail_Customer_Group_Analysis/suding/source_data/客群标签数据.xlsx')
tag_dict = pd.read_excel('/Users/suding/Desktop/Retail_Customer_Group_Analysis/suding/source_data/标签字典值.xlsx')

/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [35]:
# 构建字典映射
tag_mapping = dict(zip(tag_dict['tag_value_id'], tag_dict['tag_value_name']))
# 记录未被替换的编码值
unmatched_values = set()

In [36]:
# 替换编码值为具体取值
def replace_tag_value(value):
    if pd.isnull(value):
        return value  
    if value in tag_mapping:
        return tag_mapping[value]
    else:
        unmatched_values.add(value)
        return value  

# 遍历数据表并替换值
columns_to_replace = customer_tags.columns[3:]
for column in columns_to_replace:
    customer_tags[column] = customer_tags[column].apply(replace_tag_value)

In [37]:
# 输出未替换的编码值统计信息
print(f"共有 {len(unmatched_values)} 个编码值没有被替换:")
print(unmatched_values)

共有 32 个编码值没有被替换:
{3200, 0, 4100, 4488, 3600, 4499, 4500, 1300, 2200, 3100, 3500, 1200, 5300, 2100, 6200, 3266, 5188, 3400, 4300, 1100, 5200, 6100, 1500, 3166, 3300, 6501, 4200, 4600, 5100, 3700, 1400, 2300}


In [38]:
# 保存替换后的结果
customer_tags.to_excel('/Users/suding/Desktop/Retail_Customer_Group_Analysis/suding/source_data/替换后的客群标签数据.xlsx', index=False)

# Part 2. 拼接客户资产汇总表、客户交易汇总表、客户基本信息表

In [39]:
# 读取载入客户基本信息表、客户交易汇总表、客户资产汇总表
customer_info = pd.read_excel('/Users/suding/Desktop/Retail_Customer_Group_Analysis/suding/source_data/客户基本信息.xlsx')
customer_transactions = pd.read_excel('/Users/suding/Desktop/Retail_Customer_Group_Analysis/suding/source_data/客户交易汇总.xlsx')
customer_assets = pd.read_excel('/Users/suding/Desktop/Retail_Customer_Group_Analysis/suding/source_data/客户资产汇总.xlsx')

In [40]:
# 拼接数据集
merged_data = customer_tags.merge(customer_info, on='customer_id', how='left', suffixes=('', '_customer_info')) \
                           .merge(customer_transactions, on='customer_id', how='left', suffixes=('', '_customer_transactions')) \
                           .merge(customer_assets, on='customer_id', how='left', suffixes=('', '_customer_assets'))

In [41]:
# 检查拼接后是否有命名一致的字段
all_columns = merged_data.columns
duplicate_columns = all_columns[all_columns.duplicated()].unique()
important_duplicates = [col for col in duplicate_columns if col not in ['customer_no', 'by_month']]

# 输出命名相同的字段
if important_duplicates:
    print(f"命名相同的字段: {important_duplicates}")
else:
    print("没有命名相同的字段（忽略 customer_no 和 by_month）。")

没有命名相同的字段（忽略 customer_no 和 by_month）。


# Part 3. 处理缺失值

In [42]:
# 输出拼接后所有字段的缺失情况
missing_data = merged_data.isnull().sum()
missing_data = missing_data[missing_data > 0]
missing_data.sort_values(inplace=True, ascending=False)

In [43]:
# 记录每个字段的来源
source_columns = {
    col: 'customer_tags' for col in customer_tags.columns if col not in ['trade_date', 'customer_id', 'customer_no']
}
source_columns.update({
    col: 'customer_info' for col in customer_info.columns if col != 'customer_id'
})
source_columns.update({
    col: 'customer_transactions' for col in customer_transactions.columns if col != 'customer_id'
})
source_columns.update({
    col: 'customer_assets' for col in customer_assets.columns if col != 'customer_id'
})

missing_data_source = {col: source_columns.get(col, 'unknown') for col in missing_data.index}


In [44]:
# 输出缺失情况
print("拼接后所有字段的缺失情况及其来源:")
for col, count in missing_data.items():
    print(f"{col} (来自 {missing_data_source[col]}): 缺失 {count} 条")

拼接后所有字段的缺失情况及其来源:
end_asset_crdt (来自 customer_info): 缺失 36845 条
if_qw_active_flag (来自 customer_tags): 缺失 36845 条
level_jh_name (来自 customer_info): 缺失 36845 条
profession_name (来自 customer_info): 缺失 36845 条
begin_date_fund (来自 customer_info): 缺失 36845 条
risk_label_fund_name (来自 customer_info): 缺失 36845 条
risk_label_fund (来自 customer_info): 缺失 36845 条
begin_date_future (来自 customer_info): 缺失 36845 条
customer_kind_flag (来自 customer_info): 缺失 36845 条
registfund (来自 customer_info): 缺失 36845 条
std_date (来自 customer_info): 缺失 36845 条
status_excp_date (来自 customer_info): 缺失 36845 条
qw_qytg_push (来自 customer_tags): 缺失 36845 条
if_qw_cfzb_flag (来自 customer_tags): 缺失 36845 条
rate_normal (来自 customer_info): 缺失 36845 条
zfly (来自 customer_tags): 缺失 36845 条
if_grylcp_flag (来自 customer_tags): 缺失 36845 条
buy_pri_1y (来自 customer_tags): 缺失 36845 条
buy_qy_cust_1y (来自 customer_tags): 缺失 36845 条
buy_pub_noncur_count_1y (来自 customer_tags): 缺失 36845 条
redeem_pub_noncur_cust_7d (来自 customer_tags): 缺失 36845 条
if_p

In [45]:
# 移除完全缺失标签
fields_to_remove = [
    'end_asset_crdt', 'if_qw_active_flag', 'level_jh_name', 'profession_name', 'begin_date_fund',
    'risk_label_fund_name', 'risk_label_fund', 'begin_date_future', 'customer_kind_flag', 'registfund',
    'std_date', 'status_excp_date', 'qw_qytg_push', 'if_qw_cfzb_flag', 'rate_normal', 'zfly',
    'if_grylcp_flag', 'buy_pri_1y', 'buy_qy_cust_1y', 'buy_pub_noncur_count_1y', 'redeem_pub_noncur_cust_7d',
    'if_pension_flag', 'level_jh', 'risk_label_future', 'risk_label_future_name', 'open_id'
]
cleaned_data = merged_data.drop(columns=fields_to_remove)

# 保存移除后的结果
cleaned_data.to_excel('/Users/suding/Desktop/Retail_Customer_Group_Analysis/suding/training_data/dataset.xlsx', index=False)

In [ ]:
"""
总计十个kyc标签因缺失严重被剔除：
1. if_qw_active_flag: 
2. qw_qytg_push:
3. zfly: 
4. if_grylcp_flag:
5. buy_pri_1y: 
6. buy_qy_cust_1y: 
7. buy_pub_noncur_count_1y: 
8. redeem_pub_noncur_cust_7d: 
9. if_pension_flag: 
10.if_qw_cfzb_flag: 

"""